### Notebook to analyzse the efficiency of minimap mapping against a mock community

Starting points from Tavish  
reference_dataframe at '/media/MassStorage/tmp/TE/honours/analysis/Stats/reference_dataframe.csv'  
custom_database at '/media/MassStorage/tmp/TE/honours/database/custom_database_labelled.fasta'  
taxonomy_file at '/media/MassStorage/tmp/TE/honours/analysis/Stats/taxonomy_file.csv'

#### workflow

* Two databases
* subsample 15000 reads per each mock community species. Save those out.
* map reads against both databases with minimap safe out data in paf format.
* get best hit per species (see what this means while looking at the data).
* add the full taxonomy to each best match using the taxonomy file.
* summarize data at different taxonomic ranks for each species.
* pull this all together somehow (summary across all the samples? focus on species of interest e.g. deleted from analyis?)

#### requirment

* Bbmap (conda install bbmap https://anaconda.org/bioconda/bbmap)  
* minimap2 (conda install minimap2 https://anaconda.org/bioconda/minimap2)

#### fixes on the command line

* fixed the taxonomy file to fit with the quiime format

cat /media/MassStorage/tmp/TE/honours/analysis/Stats/taxonomy_file.csv | sed 's/,/\t/' > /media/MassStorage/tmp/TE/honours/analysis/Stats/taxonomy_file_v2.csv

#### fix the taxonomy_file_v2.csv more to reflect Qiime style

* This requires to make the species name genus_species and not _species.... hope this makes sense

In [1]:
import re
old_taxonomy_file_fn = '/media/MassStorage/tmp/TE/honours/analysis/Stats/taxonomy_file_v2.csv'
new_taxonomy_file_fn = '/media/MassStorage/tmp/TE/honours/analysis/Stats/taxonomy_file_v3.csv'
with open(new_taxonomy_file_fn, 'w') as out_fh:
    with open(old_taxonomy_file_fn, 'r') as in_fh:
        for line in in_fh:
            line = line.rstrip()
            #print(line)
            first_half = line.split('s__')[0]
            second_half = line.split('s__')[1]
            pattern = re.compile(r'g__\w+;')
            genus = re.findall(pattern, first_half)[0].replace('g__','').replace(';','')
            new_line = F"{first_half}s__{genus}_{second_half}"
            print(new_line,file=out_fh)

In [2]:
!head -2 {old_taxonomy_file_fn}

20171103_FAH15473/barcode01	k__Fungi;p__Basidiomycota;c__Pucciniomycetes;o__Pucciniales;f__Pucciniaceae;g__Puccinia;s__striiformis-tritici
20171103_FAH15473/barcode02	k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Capnodiales;f__Mycosphaerellaceae;g__Zymoseptoria;s__tritici


In [3]:
!head -2 {new_taxonomy_file_fn}

20171103_FAH15473/barcode01	k__Fungi;p__Basidiomycota;c__Pucciniomycetes;o__Pucciniales;f__Pucciniaceae;g__Puccinia;s__Puccinia_striiformis-tritici
20171103_FAH15473/barcode02	k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Capnodiales;f__Mycosphaerellaceae;g__Zymoseptoria;s__Zymoseptoria_tritici


In [4]:
from Bio import SeqIO
import os
import random
import subprocess
import pandas as pd

#### Initial data

In [5]:
reference_dataframe_fn = os.path.abspath('/media/MassStorage/tmp/TE/honours/analysis/Stats/reference_dataframe.csv')
max_custom_database_fn = os.path.abspath('/media/MassStorage/tmp/TE/honours/database/custom_database_labelled.fasta')
taxonomy_file_fn = os.path.abspath('/media/MassStorage/tmp/TE/honours/analysis/Stats/taxonomy_file_v3.csv')

In [6]:
#threads to use
threads = 6

In [7]:
INPUT_BASEDIR = os.path.abspath('/media/MassStorage/tmp/TE/honours')

In [8]:
OUT_DIR = os.path.abspath('../../analysis/Mapping_mock_gsref')
if not os.path.exists(OUT_DIR):
    os.mkdir(OUT_DIR)

In [9]:
### list of species in the max database
max_species = ['Puccinia_striiformis_tritici',
 'Zymoseptoria_tritici',
 'Pyrenophora_tritici-repentis',
 'Fusarium_oxysporum',
 'Tuber_brumale',
 'Cortinarius_globuliformis',
 'Aspergillus_niger',
 'Clavispora_lusitaniae',
 'Cryptococcus_neoformans',
 'Penicillium_chrysogenum',
 'Rhodotorula_mucilaginosa',
 'Scedosporium_boydii',
 'Blastobotrys_proliferans',
 'Candida_zeylanoides',
 'Galactomyces_geotrichum',
 'Kodamaea_ohmeri',
 'Meyerozyma_guillermondii',
 'Wickerhamomyces_anomalus',
 'Yamadazyma_mexicana',
 'Yamadazyma_scolyti',
 'Yarrowia_lipolytica',
 'Zygoascus_hellenicus',
 'Aspergillus_flavus',
 'Cryptococcus_zero',
 'Aspergillus_sp.',
 'CCL067',
 'Diaporthe_sp.',
 'Tapesia_yallundae_CCL031',
 'Tapesia_yallundae_CCL029',
 'Dothiorella_vidmadera',
 'Quambalaria_cyanescens',
 'Entoleuca_sp.',
 'CCL060',
 'CCL068',
 'Saccharomyces_cerevisiae',
 'Cladophialophora_sp.',
 'Candida_albicans',
 'Candida_metapsilosis',
 'Candida_orthopsilosis',
 'Candida_parapsilosis',
 'Geotrichum_candidum',
 'Kluyveromyces_lactis',
 'Kluyveromyces_marxianus',
 'Pichia_kudriavzevii',
 'Pichia_membranifaciens']

In [10]:
###Removed from second test databes
species_delete = ['Candida_orthopsilosis',
                 'Candida_metapsilosis',
                 'Aspergillus_niger']

In [11]:
###species to be searched against both databases
mock_community = ['Penicillium_chrysogenum',
 'Aspergillus_flavus',
 'Aspergillus_niger',
 'Pichia_kudriavzevii',
 'Pichia_membranifaciens',
 'Candida_albicans',
 'Candida_parapsilosis',
 'Candida_orthopsilosis',
 'Candida_metapsilosis']

In [12]:
fixed_old_names = ['Kluyveromyces_lactis',
                   'Candida_zeylanoides',
                   'Cladophialophora_sp.',
                   'Diaporthe_sp.',
                   'CCL060',
                   'CCL068',
                   'CCL067',
                   'Aspergillus_sp.',
                   'Entoleuca_sp.',
                   'Tapesia_yallundae_CCL029',
                   'Tapesia_yallundae_CCL031',
                   'Cryptococcus_neoformans']

In [13]:
fixed_new_names = ['candida_unidentified',
                   'debaryomyces_unidentified',
                   'cladophialophora_unidentified',
                   'diaporthe_unidentified',
                   'asteroma_ccl060',
                   'asteroma_ccl068',
                   'diaporthe_ccl067',
                   'aspergillus_unidentified',
                   'entoleuca_unidentified',
                   'oculimacula_yallundae-ccl029',
                   'oculimacula_yallundae-ccl031',
                   'kluyveromyces_unidentified']

In [14]:
old_to_new_names = dict(zip(fixed_old_names, fixed_new_names))

In [15]:
old_to_new_names

{'Kluyveromyces_lactis': 'candida_unidentified',
 'Candida_zeylanoides': 'debaryomyces_unidentified',
 'Cladophialophora_sp.': 'cladophialophora_unidentified',
 'Diaporthe_sp.': 'diaporthe_unidentified',
 'CCL060': 'asteroma_ccl060',
 'CCL068': 'asteroma_ccl068',
 'CCL067': 'diaporthe_ccl067',
 'Aspergillus_sp.': 'aspergillus_unidentified',
 'Entoleuca_sp.': 'entoleuca_unidentified',
 'Tapesia_yallundae_CCL029': 'oculimacula_yallundae-ccl029',
 'Tapesia_yallundae_CCL031': 'oculimacula_yallundae-ccl031',
 'Cryptococcus_neoformans': 'kluyveromyces_unidentified'}

### Fix databases and names

In [16]:
ref_df = pd.read_csv(reference_dataframe_fn)
ref_df['name_species'] = ref_df['genus'] +"_"+ ref_df['species']

In [17]:
ref_df.name_species.tolist()

['puccinia_striiformis-tritici',
 'zymoseptoria_tritici',
 'pyrenophora_tritici-repentis',
 'fusarium_oxysporum',
 'tuber_brumale',
 'cortinarius_globuliformis',
 'aspergillus_niger',
 'clavispora_lusitaniae',
 'kluyveromyces_unidentified',
 'penicillium_chrysogenum',
 'rhodotorula_mucilaginosa',
 'scedosporium_boydii',
 'blastobotrys_proliferans',
 'debaryomyces_unidentified',
 'galactomyces_geotrichum',
 'kodamaea_ohmeri',
 'meyerozyma_guillermondii',
 'wickerhamomyces_anomalus',
 'yamadazyma_mexicana',
 'yamadazyma_scolyti',
 'yarrowia_lipolytica',
 'zygoascus_hellenicus',
 'aspergillus_flavus',
 'cryptococcus_zero',
 'aspergillus_unidentified',
 'diaporthe_ccl067',
 'diaporthe_unidentified',
 'oculimacula_yallundae-ccl031',
 'oculimacula_yallundae-ccl029',
 'dothiorella_vidmadera',
 'quambalaria_cyanescens',
 'entoleuca_unidentified',
 'asteroma_ccl060',
 'asteroma_ccl068',
 'saccharomyces_cerevisiae',
 'cladophialophora_unidentified',
 'candida_albicans',
 'candida_metapsilosis',


In [18]:
new_db_fn = os.path.join(OUT_DIR, 'gsref.db.fasta')

In [19]:
new_db_list = []
old_db_list = []
for seq in SeqIO.parse(max_custom_database_fn, 'fasta'):
    old_db_list.append(seq.id)
    if seq.id in old_to_new_names.keys():
        #print(seq.id)
        seq.id = seq.name = seq.description = old_to_new_names[seq.id]
        new_db_list.append(seq)
    elif seq.id.lower() in ref_df.name_species.tolist():
        #print(seq.id)
        seq.id = seq.name = seq.description = seq.id.lower()
        new_db_list.append(seq)
    else:
        print(seq.id)

In [20]:
if len(new_db_list) == len(old_db_list) -2:
    SeqIO.write(new_db_list, new_db_fn, 'fasta')
else:
    print("please check!")

please check!


In [21]:
sub_db_fn = os.path.join(OUT_DIR, 'gsref.subdb.fasta')
sub_db_list = []
for seq in new_db_list:
    if seq.id not in [x.lower() for x in species_delete]:
        sub_db_list.append(seq)

In [22]:
if len(sub_db_list) + len(species_delete) == len(new_db_list):
    SeqIO.write(sub_db_list, sub_db_fn, 'fasta' )
else:
    print("please check!")

In [23]:
[x.id for x in sub_db_list]

['puccinia_striiformis-tritici',
 'zymoseptoria_tritici',
 'pyrenophora_tritici-repentis',
 'fusarium_oxysporum',
 'tuber_brumale',
 'cortinarius_globuliformis',
 'clavispora_lusitaniae',
 'kluyveromyces_unidentified',
 'penicillium_chrysogenum',
 'rhodotorula_mucilaginosa',
 'scedosporium_boydii',
 'blastobotrys_proliferans',
 'debaryomyces_unidentified',
 'galactomyces_geotrichum',
 'kodamaea_ohmeri',
 'meyerozyma_guillermondii',
 'wickerhamomyces_anomalus',
 'yamadazyma_mexicana',
 'yamadazyma_scolyti',
 'yarrowia_lipolytica',
 'zygoascus_hellenicus',
 'aspergillus_flavus',
 'cryptococcus_zero',
 'aspergillus_unidentified',
 'diaporthe_ccl067',
 'diaporthe_unidentified',
 'oculimacula_yallundae-ccl031',
 'oculimacula_yallundae-ccl029',
 'dothiorella_vidmadera',
 'quambalaria_cyanescens',
 'entoleuca_unidentified',
 'asteroma_ccl060',
 'asteroma_ccl068',
 'saccharomyces_cerevisiae',
 'cladophialophora_unidentified',
 'candida_albicans',
 'candida_parapsilosis',
 'candida_unidentified

In [24]:
mock_community = [x.lower() for x in mock_community]

In [25]:
mock_community

['penicillium_chrysogenum',
 'aspergillus_flavus',
 'aspergillus_niger',
 'pichia_kudriavzevii',
 'pichia_membranifaciens',
 'candida_albicans',
 'candida_parapsilosis',
 'candida_orthopsilosis',
 'candida_metapsilosis']

### Subsample reads

In [26]:
def subsamplereads(in_fn, out_fn, n_reads):
    command = F'reformat.sh samplereadstarget={n_reads} in={in_fn} out={out_fn}'
    out = subprocess.getstatusoutput(command)
    if out[0] == 0:
        print(F":)Completed {command}\n")
    else:
        print(F":(check one {command}!!\n")

In [27]:
n_reads = 15000

In [28]:
MC_READ_DIR = os.path.join(OUT_DIR, 'MC_READS')
if not os.path.exists(MC_READ_DIR):
    os.mkdir(MC_READ_DIR)

In [29]:
ref_df.columns

Index(['Unnamed: 0', 'species', 'genus', 'family', 'order', 'class', 'phylum',
       'kingdom', '# raw reads', '# reads after homology filtering',
       '# reads after length filtering', '# for use', 'path to raw reads',
       'path to homology filtering', 'path to length filtering',
       'path for use', 'name_species'],
      dtype='object')

In [30]:
fn_subsampling = {}
for x in mock_community:
    fn_subsampling[x] = (ref_df[(ref_df['species'] == x.split('_')[1]) & (ref_df['genus'] == x.split('_')[0])]['path for use'].tolist()[0])
    fn_subsampling[x] = os.path.join(INPUT_BASEDIR, fn_subsampling[x])
fn_subsampling

{'penicillium_chrysogenum': '/media/MassStorage/tmp/TE/honours/analysis/Length_Filtered/20171103_FAH15473/barcode10/length_restricted_for_use.fasta',
 'aspergillus_flavus': '/media/MassStorage/tmp/TE/honours/analysis/Length_Filtered/20171207_FAH18654/barcode12/length_restricted_for_use.fasta',
 'aspergillus_niger': '/media/MassStorage/tmp/TE/honours/analysis/Length_Filtered/20171103_FAH15473/barcode07/length_restricted_for_use.fasta',
 'pichia_kudriavzevii': '/media/MassStorage/tmp/TE/honours/analysis/Length_Filtered/20180108_FAH18647/barcode11/length_restricted_for_use.fasta',
 'pichia_membranifaciens': '/media/MassStorage/tmp/TE/honours/analysis/Length_Filtered/20180108_FAH18647/barcode12/length_restricted_for_use.fasta',
 'candida_albicans': '/media/MassStorage/tmp/TE/honours/analysis/Length_Filtered/20180108_FAH18647/barcode03/length_restricted_for_use.fasta',
 'candida_parapsilosis': '/media/MassStorage/tmp/TE/honours/analysis/Length_Filtered/20180108_FAH18647/barcode06/length_res

In [31]:
sub_reads_fn = {}
for key, value in fn_subsampling.items():
    species = key
    in_fn = value
    out_fn = os.path.join(MC_READ_DIR, F'{species}.{n_reads}.fasta')
    subsamplereads(in_fn, out_fn, n_reads)
    sub_reads_fn[species] = out_fn

:(check one reformat.sh samplereadstarget=15000 in=/media/MassStorage/tmp/TE/honours/analysis/Length_Filtered/20171103_FAH15473/barcode10/length_restricted_for_use.fasta out=/media/WorkingStorage/ben.working/students/tavish/analysis/Mapping_mock_gsref/MC_READS/penicillium_chrysogenum.15000.fasta!!

:(check one reformat.sh samplereadstarget=15000 in=/media/MassStorage/tmp/TE/honours/analysis/Length_Filtered/20171207_FAH18654/barcode12/length_restricted_for_use.fasta out=/media/WorkingStorage/ben.working/students/tavish/analysis/Mapping_mock_gsref/MC_READS/aspergillus_flavus.15000.fasta!!

:(check one reformat.sh samplereadstarget=15000 in=/media/MassStorage/tmp/TE/honours/analysis/Length_Filtered/20171103_FAH15473/barcode07/length_restricted_for_use.fasta out=/media/WorkingStorage/ben.working/students/tavish/analysis/Mapping_mock_gsref/MC_READS/aspergillus_niger.15000.fasta!!

:(check one reformat.sh samplereadstarget=15000 in=/media/MassStorage/tmp/TE/honours/analysis/Length_Filtered/2

### Map with minimap against both databases

In [32]:
def minimapmapping(fasta_fn, ref_fn, out_fn, threads):
    command = F"minimap2 -x map-ont -t {threads} {ref_fn} {fasta_fn} -o {out_fn}"
    out = subprocess.getstatusoutput(command)
    if out[0] == 0:
        print(F":)Completed {command}\n")
    else:
        print(F":(check one {command}!!\n")

In [33]:
dbases_fn = {}
for x in [sub_db_fn, new_db_fn]:
    dbases_fn[x] = os.path.join(OUT_DIR, os.path.basename(x).replace('.fasta', '').replace('.','_'))
    if not os.path.exists(dbases_fn[x]):
        os.mkdir(dbases_fn[x])
dbases_fn

{'/media/WorkingStorage/ben.working/students/tavish/analysis/Mapping_mock_gsref/gsref.subdb.fasta': '/media/WorkingStorage/ben.working/students/tavish/analysis/Mapping_mock_gsref/gsref_subdb',
 '/media/WorkingStorage/ben.working/students/tavish/analysis/Mapping_mock_gsref/gsref.db.fasta': '/media/WorkingStorage/ben.working/students/tavish/analysis/Mapping_mock_gsref/gsref_db'}

In [34]:
db_fn = sub_db_fn
sub_db_mapping_fn = {}
for species, fasta_fn in sub_reads_fn.items():
    tmp_out = dbases_fn[db_fn]
    db_name = os.path.basename(db_fn).replace('.fasta', '')
    out_fn = os.path.join(tmp_out, F"{db_name}.{species}.minimap2.paf")
    sub_db_mapping_fn[species] = out_fn
    minimapmapping(fasta_fn, db_fn, out_fn, threads)

:)Completed minimap2 -x map-ont -t 6 /media/WorkingStorage/ben.working/students/tavish/analysis/Mapping_mock_gsref/gsref.subdb.fasta /media/WorkingStorage/ben.working/students/tavish/analysis/Mapping_mock_gsref/MC_READS/penicillium_chrysogenum.15000.fasta -o /media/WorkingStorage/ben.working/students/tavish/analysis/Mapping_mock_gsref/gsref_subdb/gsref.subdb.penicillium_chrysogenum.minimap2.paf

:)Completed minimap2 -x map-ont -t 6 /media/WorkingStorage/ben.working/students/tavish/analysis/Mapping_mock_gsref/gsref.subdb.fasta /media/WorkingStorage/ben.working/students/tavish/analysis/Mapping_mock_gsref/MC_READS/aspergillus_flavus.15000.fasta -o /media/WorkingStorage/ben.working/students/tavish/analysis/Mapping_mock_gsref/gsref_subdb/gsref.subdb.aspergillus_flavus.minimap2.paf

:)Completed minimap2 -x map-ont -t 6 /media/WorkingStorage/ben.working/students/tavish/analysis/Mapping_mock_gsref/gsref.subdb.fasta /media/WorkingStorage/ben.working/students/tavish/analysis/Mapping_mock_gsref/M

In [35]:
db_fn = new_db_fn
new_db_mapping_fn = {}
for species, fasta_fn in sub_reads_fn.items():
    tmp_out = dbases_fn[db_fn]
    db_name = os.path.basename(db_fn).replace('.fasta', '')
    out_fn = os.path.join(tmp_out, F"{db_name}.{species}.minimap2.paf")
    new_db_mapping_fn[species] = out_fn
    minimapmapping(fasta_fn, db_fn, out_fn, threads)

:)Completed minimap2 -x map-ont -t 6 /media/WorkingStorage/ben.working/students/tavish/analysis/Mapping_mock_gsref/gsref.db.fasta /media/WorkingStorage/ben.working/students/tavish/analysis/Mapping_mock_gsref/MC_READS/penicillium_chrysogenum.15000.fasta -o /media/WorkingStorage/ben.working/students/tavish/analysis/Mapping_mock_gsref/gsref_db/gsref.db.penicillium_chrysogenum.minimap2.paf

:)Completed minimap2 -x map-ont -t 6 /media/WorkingStorage/ben.working/students/tavish/analysis/Mapping_mock_gsref/gsref.db.fasta /media/WorkingStorage/ben.working/students/tavish/analysis/Mapping_mock_gsref/MC_READS/aspergillus_flavus.15000.fasta -o /media/WorkingStorage/ben.working/students/tavish/analysis/Mapping_mock_gsref/gsref_db/gsref.db.aspergillus_flavus.minimap2.paf

:)Completed minimap2 -x map-ont -t 6 /media/WorkingStorage/ben.working/students/tavish/analysis/Mapping_mock_gsref/gsref.db.fasta /media/WorkingStorage/ben.working/students/tavish/analysis/Mapping_mock_gsref/MC_READS/aspergillus_n

### Look at mapping results

In [36]:
def mapping_results(fn, species):
    min_header = ['qseqid', 'qlen', 'qstart', 'qstop', 'strand', 'tname', 'tlen', 'tstart', 'tend', 'nmatch', 'alen', 'mquality']
    tmp_df = pd.read_csv(fn, sep='\t', header = None, usecols=[x for x in range(0,12)], names=min_header)
    sub_df = tmp_df[tmp_df['mquality'] == tmp_df.groupby('qseqid')['mquality'].transform(max)].reset_index(drop=True)
    sub_df = sub_df[sub_df['nmatch'] == sub_df.groupby('qseqid')['nmatch'].transform(max)].reset_index(drop=True)
    hit_series = pd.Series(sub_df.groupby('tname')['mquality'].count().tolist()/sub_df.groupby('tname')['mquality'].count().sum(),
                      sub_df.groupby('tname')['mquality'].count().index)
    hit_series.sort_values(ascending=False, inplace=True)
    print(sub_df.qseqid.unique().shape == tmp_df.qseqid.unique().shape)
    print('##########\n')
    print(F"This was the query species: {species}\n")
    print(F"These are the results:")
    print(hit_series,'\n')

In [37]:
###this is running the reads against the full database
for species, hit_fn in new_db_mapping_fn.items():
    mapping_results(hit_fn, species)

True
##########

This was the query species: penicillium_chrysogenum

These are the results:
tname
penicillium_chrysogenum          0.992277
aspergillus_niger                0.001798
aspergillus_flavus               0.001398
aspergillus_unidentified         0.000932
tuber_brumale                    0.000666
kluyveromyces_marxianus          0.000466
zymoseptoria_tritici             0.000466
pyrenophora_tritici-repentis     0.000333
kluyveromyces_unidentified       0.000266
puccinia_striiformis-tritici     0.000200
cortinarius_globuliformis        0.000133
asteroma_ccl060                  0.000133
clavispora_lusitaniae            0.000133
scedosporium_boydii              0.000067
rhodotorula_mucilaginosa         0.000067
asteroma_ccl068                  0.000067
cladophialophora_unidentified    0.000067
pichia_membranifaciens           0.000067
oculimacula_yallundae-ccl031     0.000067
diaporthe_unidentified           0.000067
fusarium_oxysporum               0.000067
kodamaea_ohmeri    

True
##########

This was the query species: candida_orthopsilosis

These are the results:
tname
candida_orthopsilosis            0.734469
candida_metapsilosis             0.204357
candida_parapsilosis             0.041606
meyerozyma_guillermondii         0.010132
candida_unidentified             0.001330
candida_albicans                 0.000887
tuber_brumale                    0.000633
galactomyces_geotrichum          0.000570
kodamaea_ohmeri                  0.000507
yamadazyma_scolyti               0.000507
saccharomyces_cerevisiae         0.000507
debaryomyces_unidentified        0.000443
clavispora_lusitaniae            0.000380
entoleuca_unidentified           0.000380
oculimacula_yallundae-ccl029     0.000380
yarrowia_lipolytica              0.000317
zygoascus_hellenicus             0.000317
cladophialophora_unidentified    0.000253
yamadazyma_mexicana              0.000253
blastobotrys_proliferans         0.000190
kluyveromyces_marxianus          0.000190
puccinia_striiformis-

In [38]:
###this is running against a database that have ['Candida_orthopsilosis', 'Candida_metapsilosis', 'Aspergillus_niger'] deleted
for species, hit_fn in sub_db_mapping_fn.items():
    mapping_results(hit_fn, species)

True
##########

This was the query species: penicillium_chrysogenum

These are the results:
tname
penicillium_chrysogenum          0.993337
aspergillus_unidentified         0.001732
aspergillus_flavus               0.001333
tuber_brumale                    0.000666
kluyveromyces_marxianus          0.000466
zymoseptoria_tritici             0.000466
pyrenophora_tritici-repentis     0.000333
kluyveromyces_unidentified       0.000267
puccinia_striiformis-tritici     0.000200
cortinarius_globuliformis        0.000133
asteroma_ccl060                  0.000133
oculimacula_yallundae-ccl031     0.000133
clavispora_lusitaniae            0.000133
rhodotorula_mucilaginosa         0.000067
asteroma_ccl068                  0.000067
cladophialophora_unidentified    0.000067
pichia_membranifaciens           0.000067
cryptococcus_zero                0.000067
fusarium_oxysporum               0.000067
kodamaea_ohmeri                  0.000067
meyerozyma_guillermondii         0.000067
oculimacula_yallund

### Pull in mapping results and analyse them at all available levels

##### idea

* pull in query taxid as a dictionary
* assign taxid for each tname species from minimap2
* generate a summary dictionary that checks concordance at each taxonmic rank

In [39]:
def pull_mapping_results(fn):
    """
    Takes a minimap2 paf and reads it in with the first 12 columns. Ignores the rest.
    Filters for each read the best hit on mquality first taking the highest value.
    Filters for each read by the number of nmatches in the second step.
    Returns a dataframe that has the tnames as index and the counts of hits as column 'count'.
    The dataframe has also the taxrank columns ['k', 'p', 'c', 'o', 'f', 'g', 's'] that are all False to start with.
    """
    min_header = ['qseqid', 'qlen', 'qstart', 'qstop', 'strand', 'tname', 'tlen', 'tstart', 'tend', 'nmatch', 'alen', 'mquality']
    tmp_df = pd.read_csv(fn, sep='\t', header = None, usecols=[x for x in range(0,12)], names=min_header)
    sub_df = tmp_df[tmp_df['mquality'] == tmp_df.groupby('qseqid')['mquality'].transform(max)].reset_index(drop=True)
    sub_df = sub_df[sub_df['nmatch'] == sub_df.groupby('qseqid')['nmatch'].transform(max)].reset_index(drop=True)
    hit_df = pd.DataFrame(sub_df.groupby('tname')['mquality'].count().tolist(), sub_df.groupby('tname')['mquality'].count().index, columns=['count'])
    hit_df.sort_values(by='count', ascending=False, inplace=True)
    for key in ['k', 'p', 'c', 'o', 'f', 'g', 's']:
        hit_df[key] = False
    return hit_df

In [40]:
def getquery_taxfileid(refdf_fn, species):
    """
    Takes the reference dataframe filename and the species name.
    Returns the taxfileid, which is the date/flowcellid (column 0 value) of the ref_df.
    """
    ref_df = pd.read_csv(refdf_fn)
    ref_df['name_species'] = ref_df['genus'] +"_"+ ref_df['species']
    return ref_df[ref_df.name_species == species].iloc[:,0].values[0]

In [41]:
def get_taxid_dict(taxid_fn, taxfileid):
    """
    Takes a taxonomy assignment file filename in the Qiime format and a taxonomic identifier.
    Returns the a dictionary with the taxonomic assignment at each rank.
    """
    tax_dict = {}
    with open(taxid_fn, 'r') as fh:
        for line in fh:
            if line.startswith(taxfileid):
                taxrankids = line.rstrip().split('\t')[1].split(';')
                for taxrank in taxrankids:
                    tax_dict[taxrank.split('__')[0]] = taxrank.split('__')[1]
    return tax_dict

In [42]:
def assign_taxranks_results(mapping_df, tax_fn, ref_df_fn = False):
    """
    This function assigns the taxonomic ranks for each hit in the mapping results dataframe.
    It takes a mapping_df, taxonomy assignment file, and if required a reference dataframe filename.
    Returns the mapping dataframe with assignment. 
    """
    for tname in mapping_df.index:
        if ref_df_fn:
            tmp_taxfileid = getquery_taxfileid(ref_df_fn, tname)
        else:
            tmp_taxfileid = tname
        tmp_tax_dict = get_taxid_dict(tax_fn, tmp_taxfileid)
        for key, value in tmp_tax_dict.items():
            mapping_df.loc[tname, key] = value
    return mapping_df

In [43]:
def get_accuracy_dict(mapping_df, query_tax_dict):
    """
    Summarieses the mapping accuracy of the mapping results at all taxonomic ranks.
    Takes the mapping_df with taxnomonic assignments and a taxnomic dictionary of the known query.
    Returns an accuracy dictionary for each taxnomic rank ['k', 'p', 'c', 'o', 'f', 'g', 's']. 
    Right now this function takes a qiime tax 
    """
    accuracy_dict = {}
    total_count = mapping_df['count'].sum()
    for tax_rank in ['k', 'p', 'c', 'o', 'f', 'g', 's']:
        hit_count = mapping_df[mapping_df[tax_rank] == query_tax_dict[tax_rank]]['count'].sum()
            
        accuracy_dict[tax_rank] = hit_count/total_count
    return accuracy_dict

In [44]:
###Test out the summary results statistic for a single mapping result
species = 'penicillium_chrysogenum'
mapping_results = pull_mapping_results(sub_db_mapping_fn[species])

In [45]:
###Assign the data taxonomics ranks for all the results
mapping_results = assign_taxranks_results(mapping_results, taxonomy_file_fn, ref_df_fn=reference_dataframe_fn)

taxfileid = getquery_taxfileid(reference_dataframe_fn, species)

query_tax_dict = get_taxid_dict(taxonomy_file_fn, taxfileid)

sensitivity_dict = get_accuracy_dict(mapping_results, query_tax_dict)

In [46]:
sensitivity_dict

{'k': 1.0,
 'p': 0.9995335820895522,
 'c': 0.9964685501066098,
 'o': 0.9964019189765458,
 'f': 0.9964019189765458,
 'g': 0.9933368869936035,
 's': 0.9933368869936035}

### Test run on the quiime2 Database

##### Prep on the command line

cp sh_refs_qiime_ver8_dynamic_02.02.2019.fasta /media/WorkingStorage/ben.working/students/tavish/analysis/qiime2/db/.  
cp sh_taxonomy_qiime_ver8_dynamic_02.02.2019.txt /media/WorkingStorage/ben.working/students/tavish/analysis/qiime2/db/.


In [90]:
def pull_mapping_results_v2(fn):
    """
    Takes a minimap2 paf and reads it in with the first 12 columns. Ignores the rest.
    Filters for each read the best hit on mquality first taking the highest value.
    Filters for each read by the number of nmatches in the second step.
    Returns a dataframe that has the tnames as index and the counts of hits as column 'count'.
    The dataframe has also the taxrank columns ['k', 'p', 'c', 'o', 'f', 'g', 's'] that are all False to start with.
    """
    min_header = ['qseqid', 'qlen', 'qstart', 'qstop', 'strand', 'tname', 'tlen', 'tstart', 'tend', 'nmatch', 'alen', 'mquality']
    tmp_df = pd.read_csv(fn, sep='\t', header = None, usecols=[x for x in range(0,12)], names=min_header)
    sub_df = tmp_df[tmp_df['mquality'] == tmp_df.groupby('qseqid')['mquality'].transform(max)].reset_index(drop=True)
    #sub_df = sub_df[sub_df['nmatch'] == sub_df.groupby('qseqid')['nmatch'].transform(max)].reset_index(drop=True)
    hit_df = pd.DataFrame(sub_df.groupby('tname')['mquality'].count().tolist(), sub_df.groupby('tname')['mquality'].count().index, columns=['count'])
    hit_df.sort_values(by='count', ascending=False, inplace=True)
    for key in ['k', 'p', 'c', 'o', 'f', 'g', 's']:
        hit_df[key] = False
        tmp_df[key] = False
    return hit_df, tmp_df

In [48]:
os.path.abspath(os.curdir)

'/media/WorkingStorage/ben.working/students/tavish/scripts/notebooks'

In [49]:
qiime_db_fn = os.path.abspath('../../analysis/qiime2/db/sh_refs_qiime_ver8_dynamic_02.02.2019.fasta')
qiime_tax_fn = os.path.abspath('../../analysis/qiime2/db/sh_taxonomy_qiime_ver8_dynamic_02.02.2019.txt')
threads = 10
QIIME_DIR = os.path.abspath('../../analysis/qiime2/')

In [50]:
##mapping folder
mapping_dir = os.path.join(QIIME_DIR, os.path.basename(qiime_db_fn).replace('.fasta', '').replace('.','_'))
if not os.path.exists(mapping_dir):
    os.mkdir(mapping_dir)
subsampling_dir = os.path.join(QIIME_DIR, 'subsamplereads')
if not os.path.exists(subsampling_dir):
    os.mkdir(subsampling_dir)

#### Run on test species 'penicillium_chrysogenum'

In [93]:
#subsample tests species
fn_subsampling = {}
test_species = ['penicillium_chrysogenum']
for x in test_species:
    fn_subsampling[x] = (ref_df[(ref_df['species'] == x.split('_')[1]) & (ref_df['genus'] == x.split('_')[0])]['path for use'].tolist()[0])
    fn_subsampling[x] = os.path.join(INPUT_BASEDIR, fn_subsampling[x])

sub_reads_fn = {}
n_reads = 20000
for key, value in fn_subsampling.items():
    species = key
    in_fn = value
    out_fn = os.path.join(subsampling_dir, F'{species}.{n_reads}.fasta')
    subsamplereads(in_fn, out_fn, n_reads)
    sub_reads_fn[species] = out_fn

###Map the reads
db_fn = qiime_db_fn
sub_db_mapping_fn = {}
for species, fasta_fn in sub_reads_fn.items():
    db_name = os.path.basename(db_fn).replace('.fasta', '')
    out_fn = os.path.join(mapping_dir, F"{db_name}.{species}.minimap2.paf")
    sub_db_mapping_fn[species] = out_fn
    minimapmapping(fasta_fn, db_fn, out_fn, threads)

###Test out the summary results statistic for a single mapping result
species = 'penicillium_chrysogenum'
mapping_results , full_results_df = pull_mapping_results_v2(sub_db_mapping_fn[species])
mapping_results = assign_taxranks_results(mapping_results, qiime_tax_fn)
taxfileid = getquery_taxfileid(reference_dataframe_fn, species)
query_tax_dict = get_taxid_dict(taxonomy_file_fn, taxfileid)
###fix family level for 'penicillium_chrysogenum'
query_tax_dict['f'] = 'Trichomonascaceae'
sensitivity_dict = get_accuracy_dict(mapping_results, query_tax_dict)


full_results_df.index = full_results_df.tname
###Also look at the full results dataframe to explore results a bit more
for tname in full_results_df.tname.unique():

    tmp_tax_dict = get_taxid_dict(qiime_tax_fn, tname)
    for key, value in tmp_tax_dict.items():
        full_results_df.loc[tname, key] = value

:(check one reformat.sh samplereadstarget=20000 in=/media/MassStorage/tmp/TE/honours/analysis/Length_Filtered/20171103_FAH15473/barcode10/length_restricted_for_use.fasta out=/media/WorkingStorage/ben.working/students/tavish/analysis/qiime2/subsamplereads/penicillium_chrysogenum.20000.fasta!!

:)Completed minimap2 -x map-ont -t 10 /media/WorkingStorage/ben.working/students/tavish/analysis/qiime2/db/sh_refs_qiime_ver8_99_02.02.2019.fasta /media/WorkingStorage/ben.working/students/tavish/analysis/qiime2/subsamplereads/penicillium_chrysogenum.20000.fasta -o /media/WorkingStorage/ben.working/students/tavish/analysis/qiime2/sh_refs_qiime_ver8_99_02_02_2019/sh_refs_qiime_ver8_99_02.02.2019.penicillium_chrysogenum.minimap2.paf



In [85]:
sensitivity_dict 

{'k': 1.0,
 'p': 0.8811273411596021,
 'c': 0.18675508558114526,
 'o': 0.18594856169907698,
 'f': 0.004391074469038444,
 'g': 0.08275831167667354,
 's': 0.003987812528004301}

In [89]:
full_results_df

,qseqid,qlen,qstart,qstop,strand,tname,tlen,tstart,tend,nmatch,alen,mquality,k,p,c,o,f,g,s,count
tname,,,,,,,,,,,,,,,,,,,,
SH1649638.08FU_UDB016769_refs,fa3febf1-c473-442a-b9be-d8124c4dfd86,2927,165,514,-,SH1649638.08FU_UDB016769_refs,1720,1270,1639,68,370,18,Fungi,Basidiomycota,Agaricomycetes,Thelephorales,Thelephoraceae,Tomentella,unidentified,1
SH1654757.08FU_UDB014954_reps,fa3febf1-c473-442a-b9be-d8124c4dfd86,2927,343,803,-,SH1654757.08FU_UDB014954_reps,1390,812,1293,68,482,28,Fungi,Ascomycota,Orbiliomycetes,Orbiliales,unidentified,unidentified,unidentified,1
SH1680040.08FU_KC992945_refs_singleton,fa3febf1-c473-442a-b9be-d8124c4dfd86,2927,343,514,-,SH1680040.08FU_KC992945_refs_singleton,1573,1280,1466,53,187,0,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Agaricaceae,Cystoagaricus,Cystoagaricus_hirtosquamulosus,1
SH1566136.08FU_KC992951_refs,fa3febf1-c473-442a-b9be-d8124c4dfd86,2927,343,514,-,SH1566136.08FU_KC992951_refs,1588,1296,1482,53,187,0,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Psathyrellaceae,Lacrymaria,Lacrymaria_subcinnamomea,1
SH1896017.08FU_AF034454_refs,fa3febf1-c473-442a-b9be-d8124c4dfd86,2927,1448,1518,-,SH1896017.08FU_AF034454_refs,506,126,193,41,70,1,Fungi,Ascomycota,Eurotiomycetes,Eurotiales,Aspergillaceae,Penicillium,Penicillium_turbatum,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SH2190163.08FU_AF033485_refs,ed39c3c6-0c0f-4ab6-af60-a6af445b7ac0,2983,1538,1684,-,SH2190163.08FU_AF033485_refs,505,15,159,45,147,0,Fungi,Ascomycota,Eurotiomycetes,Eurotiales,Aspergillaceae,Penicillium,Penicillium_malodoratum,1
SH2189971.08FU_JX997101_refs,ed39c3c6-0c0f-4ab6-af60-a6af445b7ac0,2983,1538,1684,-,SH2189971.08FU_JX997101_refs,496,4,149,45,148,0,Fungi,Ascomycota,Eurotiomycetes,Eurotiales,Aspergillaceae,Penicillium,Penicillium_nalgiovense,1
SH1654757.08FU_UDB014954_reps,ed3f13ef-aae3-44b4-b5b3-ec4c103e4b44,2750,1678,2396,+,SH1654757.08FU_UDB014954_reps,1390,519,1282,59,763,12,Fungi,Ascomycota,Orbiliomycetes,Orbiliales,unidentified,unidentified,unidentified,1


In [87]:
###looking at unfiltered results
###look at the results unfiltered
full_results_df['count'] = 1

get_accuracy_dict(full_results_df, query_tax_dict)

{'k': 1.0,
 'p': 0.9110033551294296,
 'c': 0.4338953787377856,
 'o': 0.4282749246932628,
 'f': 0.004114319300565719,
 'g': 0.38500722454877184,
 's': 0.030410452330223103}

##### These are wired results that might be linked to

* database issues as you can see

sh_taxonomy_qiime_ver8_dynamic_02.02.2019.txt: k__Fungi;p__Ascomycota;c__Eurotiomycetes;o__Eurotiales;f__Aspergillaceae;g__Penicillium;s__Penicillium_chrysogenum

In [88]:
mapping_results[mapping_results['s'] == 'Penicillium_chrysogenum']

,count,k,p,c,o,f,g,s
tname,,,,,,,,
SH1530047.08FU_HE649392_reps,50,Fungi,Ascomycota,Eurotiomycetes,Eurotiales,Aspergillaceae,Penicillium,Penicillium_chrysogenum
SH2189908.08FU_AY213669_refs,39,Fungi,Ascomycota,Eurotiomycetes,Eurotiales,Aspergillaceae,Penicillium,Penicillium_chrysogenum


In [60]:
query_tax_dict = {}
taxrank = 'k__Fungi;p__Ascomycota;c__Eurotiomycetes;o__Eurotiales;f__Aspergillaceae;g__Penicillium;s__Penicillium_chrysogenum'
for rank_id in taxrank.split(';'):
    query_tax_dict[rank_id.split('__')[0]] = rank_id.split('__')[1]
query_tax_dict

{'k': 'Fungi',
 'p': 'Ascomycota',
 'c': 'Eurotiomycetes',
 'o': 'Eurotiales',
 'f': 'Aspergillaceae',
 'g': 'Penicillium',
 's': 'Penicillium_chrysogenum'}

In [61]:
get_accuracy_dict(mapping_results, query_tax_dict)

{'k': 1.0,
 'p': 0.8811273411596021,
 'c': 0.18675508558114526,
 'o': 0.18594856169907698,
 'f': 0.18505242405233444,
 'g': 0.08275831167667354,
 's': 0.003987812528004301}

In [62]:
###There must be more or equal number of mapping results compared to number of mapped reads
mapping_results['count'].sum() >= full_results_df.qseqid.unique().shape[0]

True

#### Testing on Candida albicans

In [63]:
#subsample tests species
fn_subsampling = {}
test_species = ['candida_albicans']
for x in test_species:
    fn_subsampling[x] = (ref_df[(ref_df['species'] == x.split('_')[1]) & (ref_df['genus'] == x.split('_')[0])]['path for use'].tolist()[0])
    fn_subsampling[x] = os.path.join(INPUT_BASEDIR, fn_subsampling[x])

sub_reads_fn = {}
n_reads = 20000
for key, value in fn_subsampling.items():
    species = key
    in_fn = value
    out_fn = os.path.join(subsampling_dir, F'{species}.{n_reads}.fasta')
    subsamplereads(in_fn, out_fn, n_reads)
    sub_reads_fn[species] = out_fn

:(check one reformat.sh samplereadstarget=20000 in=/media/MassStorage/tmp/TE/honours/analysis/Length_Filtered/20180108_FAH18647/barcode03/length_restricted_for_use.fasta out=/media/WorkingStorage/ben.working/students/tavish/analysis/qiime2/subsamplereads/candida_albicans.20000.fasta!!



In [64]:
###Map the reads
db_fn = qiime_db_fn
sub_db_mapping_fn = {}
for species, fasta_fn in sub_reads_fn.items():
    db_name = os.path.basename(db_fn).replace('.fasta', '')
    out_fn = os.path.join(mapping_dir, F"{db_name}.{species}.minimap2.paf")
    sub_db_mapping_fn[species] = out_fn
    minimapmapping(fasta_fn, db_fn, out_fn, threads)

:)Completed minimap2 -x map-ont -t 10 /media/WorkingStorage/ben.working/students/tavish/analysis/qiime2/db/sh_refs_qiime_ver8_dynamic_02.02.2019.fasta /media/WorkingStorage/ben.working/students/tavish/analysis/qiime2/subsamplereads/candida_albicans.20000.fasta -o /media/WorkingStorage/ben.working/students/tavish/analysis/qiime2/sh_refs_qiime_ver8_dynamic_02_02_2019/sh_refs_qiime_ver8_dynamic_02.02.2019.candida_albicans.minimap2.paf



In [65]:
###Test out the summary results statistic for a single mapping result
species = test_species[0]
print(sub_db_mapping_fn[species])
mapping_results , full_results_df = pull_mapping_results_v2(sub_db_mapping_fn[species])
mapping_results = assign_taxranks_results(mapping_results, qiime_tax_fn)
taxfileid = getquery_taxfileid(reference_dataframe_fn, species)
query_tax_dict = get_taxid_dict(taxonomy_file_fn, taxfileid)

sensitivity_dict = get_accuracy_dict(mapping_results, query_tax_dict)

###Also look at the full results dataframe to explore results a bit more
#for tname in full_results_df.tname.unique():
    #tmp_tax_dict = get_taxid_dict(qiime_tax_fn, tname)
    #for key, value in tmp_tax_dict.items():
        #full_results_df.loc[tname, key] = value

/media/WorkingStorage/ben.working/students/tavish/analysis/qiime2/sh_refs_qiime_ver8_dynamic_02_02_2019/sh_refs_qiime_ver8_dynamic_02.02.2019.candida_albicans.minimap2.paf


In [66]:
sensitivity_dict

{'k': 1.0,
 'p': 0.8180013515796587,
 'c': 0.5216252745396182,
 'o': 0.5216252745396182,
 'f': 0.07247845919918905,
 'g': 0.44749957763135667,
 's': 0.4208058793715155}

#### Might want to double check how your families are matched in your taxonomic input file for your known test species

In [67]:
mapping_results[mapping_results['s'] == 'Candida_albicans']

,count,k,p,c,o,f,g,s
tname,,,,,,,,
SH1570378.08FU_MF767825_reps,7823,Fungi,Ascomycota,Saccharomycetes,Saccharomycetales,Saccharomycetales_fam_Incertae_sedis,Candida,Candida_albicans
SH1570374.08FU_EF567995_refs,2102,Fungi,Ascomycota,Saccharomycetes,Saccharomycetales,Saccharomycetales_fam_Incertae_sedis,Candida,Candida_albicans
SH1570379.08FU_KP675460_reps,16,Fungi,Ascomycota,Saccharomycetes,Saccharomycetales,Saccharomycetales_fam_Incertae_sedis,Candida,Candida_albicans
SH1570376.08FU_GQ280331_reps,11,Fungi,Ascomycota,Saccharomycetes,Saccharomycetales,Saccharomycetales_fam_Incertae_sedis,Candida,Candida_albicans
SH1570377.08FU_KP674963_reps,11,Fungi,Ascomycota,Saccharomycetes,Saccharomycetales,Saccharomycetales_fam_Incertae_sedis,Candida,Candida_albicans


In [68]:
query_tax_dict['f'] = 'Saccharomycetales_fam_Incertae_sedis'

In [69]:
get_accuracy_dict(mapping_results, query_tax_dict)

{'k': 1.0,
 'p': 0.8180013515796587,
 'c': 0.5216252745396182,
 'o': 0.5216252745396182,
 'f': 0.44749957763135667,
 'g': 0.44749957763135667,
 's': 0.4208058793715155}

### It appears that the filtering of results by mapping quality works well for the long ITS database but not for the quiime

In [70]:
full_results_df.columns

Index(['qseqid', 'qlen', 'qstart', 'qstop', 'strand', 'tname', 'tlen',
       'tstart', 'tend', 'nmatch', 'alen', 'mquality', 'k', 'p', 'c', 'o', 'f',
       'g', 's'],
      dtype='object')

In [71]:
full_results_df.tname.shape

(133263,)

In [72]:
full_results_df.tname.unique().shape

(166,)

In [73]:
full_results_df.index = full_results_df.tname

In [74]:
###Asign taxonomic rangs to the full_results_df
for tname in full_results_df.tname.unique():
    tmp_tax_dict = get_taxid_dict(qiime_tax_fn, tname)
    for key, value in tmp_tax_dict.items():
        full_results_df.loc[tname, key] = value

In [75]:
full_results_df[full_results_df['g'] == 'Candida']['qseqid'].shape

(28996,)

In [76]:
full_results_df[full_results_df['g'] == 'Candida']['qseqid'].unique().shape

(18020,)

In [77]:
###look at the results unfiltered
full_results_df['count'] = 1

get_accuracy_dict(full_results_df, query_tax_dict)

{'k': 1.0,
 'p': 0.8166107621770484,
 'c': 0.29843992706152495,
 'o': 0.29843992706152495,
 'f': 0.21758477596932382,
 'g': 0.21758477596932382,
 's': 0.17861672031996878}

Looking at the results unfiltered doesn't really work very well either. Might need to look into different filtering of the alignments or the qiime2 database might be just not really useful for the noise reads. Simulated reads with higher accuracy should get better here.

In [78]:
full_results_df.groupby('g').count()['k'].sort_values(ascending=False)

g
Candida                28996
Acidea                 16895
Plectosphaerella       12211
unidentified           10961
Nakaseomyces           10270
Oidiodendron            9451
Tomentella              8991
Knoxdaviesia            7543
Lignincola              6168
Cystoagaricus           5014
Lacrymaria              4981
Bannoa                  4605
Pseudorobillarda        2802
Deltopyxis               861
Pestalotiopsis           699
Leveillula               684
Wickerhamiella           411
Orpinomyces              372
Exophiala                144
Phyllactinia             136
Physcia                  130
Sporisorium              129
Pyrenodesmia             126
Physconia                118
Rhinocladiella           106
Pseudophaeomoniella      103
Sclerophora              101
Conioscypha               38
Chromocleista             37
Fusarium                  33
Rhodotorula               28
Xylaria                   18
Lodderomyces              16
Scheffersomyces           16
Spathaspora 

In [79]:
###There must be more or equal number of mapping results compared to number of mapped reads
mapping_results['count'].sum() >= full_results_df.qseqid.unique().shape[0]

True

### Test run on the quiime2 Database 99

##### Prep on the command line

cp sh_refs_qiime_ver8_99_02.02.2019.fasta /media/WorkingStorage/ben.working/students/tavish/analysis/qiime2/db/. 
cp sh_taxonomy_qiime_ver8_99_02.02.2019.txt /media/WorkingStorage/ben.working/students/tavish/analysis/qiime2/db/.

In [91]:
qiime_db_fn = os.path.abspath('../../analysis/qiime2/db/sh_refs_qiime_ver8_99_02.02.2019.fasta')
qiime_tax_fn = os.path.abspath('../../analysis/qiime2/db/sh_taxonomy_qiime_ver8_99_02.02.2019.txt')
threads = 10
QIIME_DIR = os.path.abspath('../../analysis/qiime2/')

In [92]:
##mapping folder
mapping_dir = os.path.join(QIIME_DIR, os.path.basename(qiime_db_fn).replace('.fasta', '').replace('.','_'))
if not os.path.exists(mapping_dir):
    os.mkdir(mapping_dir)
subsampling_dir = os.path.join(QIIME_DIR, 'subsamplereads')
if not os.path.exists(subsampling_dir):
    os.mkdir(subsampling_dir)

In [94]:
#subsample tests species
fn_subsampling = {}
test_species = ['penicillium_chrysogenum']
for x in test_species:
    fn_subsampling[x] = (ref_df[(ref_df['species'] == x.split('_')[1]) & (ref_df['genus'] == x.split('_')[0])]['path for use'].tolist()[0])
    fn_subsampling[x] = os.path.join(INPUT_BASEDIR, fn_subsampling[x])

sub_reads_fn = {}
n_reads = 20000
for key, value in fn_subsampling.items():
    species = key
    in_fn = value
    out_fn = os.path.join(subsampling_dir, F'{species}.{n_reads}.fasta')
    subsamplereads(in_fn, out_fn, n_reads)
    sub_reads_fn[species] = out_fn

###Map the reads
db_fn = qiime_db_fn
sub_db_mapping_fn = {}
for species, fasta_fn in sub_reads_fn.items():
    db_name = os.path.basename(db_fn).replace('.fasta', '')
    out_fn = os.path.join(mapping_dir, F"{db_name}.{species}.minimap2.paf")
    sub_db_mapping_fn[species] = out_fn
    minimapmapping(fasta_fn, db_fn, out_fn, threads)

###Test out the summary results statistic for a single mapping result
species = 'penicillium_chrysogenum'
mapping_results , full_results_df = pull_mapping_results_v2(sub_db_mapping_fn[species])
mapping_results = assign_taxranks_results(mapping_results, qiime_tax_fn)
taxfileid = getquery_taxfileid(reference_dataframe_fn, species)
query_tax_dict = get_taxid_dict(taxonomy_file_fn, taxfileid)
###fix family level for 'penicillium_chrysogenum'
query_tax_dict['f'] = 'Trichomonascaceae'
sensitivity_dict = get_accuracy_dict(mapping_results, query_tax_dict)


full_results_df.index = full_results_df.tname
###Also look at the full results dataframe to explore results a bit more
for tname in full_results_df.tname.unique():

    tmp_tax_dict = get_taxid_dict(qiime_tax_fn, tname)
    for key, value in tmp_tax_dict.items():
        full_results_df.loc[tname, key] = value

:(check one reformat.sh samplereadstarget=20000 in=/media/MassStorage/tmp/TE/honours/analysis/Length_Filtered/20171103_FAH15473/barcode10/length_restricted_for_use.fasta out=/media/WorkingStorage/ben.working/students/tavish/analysis/qiime2/subsamplereads/penicillium_chrysogenum.20000.fasta!!

:)Completed minimap2 -x map-ont -t 10 /media/WorkingStorage/ben.working/students/tavish/analysis/qiime2/db/sh_refs_qiime_ver8_99_02.02.2019.fasta /media/WorkingStorage/ben.working/students/tavish/analysis/qiime2/subsamplereads/penicillium_chrysogenum.20000.fasta -o /media/WorkingStorage/ben.working/students/tavish/analysis/qiime2/sh_refs_qiime_ver8_99_02_02_2019/sh_refs_qiime_ver8_99_02.02.2019.penicillium_chrysogenum.minimap2.paf



In [95]:
sensitivity_dict

{'k': 1.0,
 'p': 0.7808393002809753,
 'c': 0.26216804133055377,
 'o': 0.2609444394090456,
 'f': 0.005030363455089277,
 'g': 0.10237469409951962,
 's': 4.5318589685488985e-05}

In [96]:
###looking at unfiltered results
###look at the results unfiltered
full_results_df['count'] = 1

get_accuracy_dict(full_results_df, query_tax_dict)

{'k': 1.0,
 'p': 0.837235656449343,
 'c': 0.3831831502501021,
 'o': 0.37726112360098213,
 'f': 0.004941114827609317,
 'g': 0.29803878561897973,
 's': 7.311144011259162e-05}

In [99]:
###fix the query_tax_dict with the data found in the qiime database
query_tax_dict = {}
taxrank = 'k__Fungi;p__Ascomycota;c__Eurotiomycetes;o__Eurotiales;f__Aspergillaceae;g__Penicillium;s__Penicillium_chrysogenum'
for rank_id in taxrank.split(';'):
    query_tax_dict[rank_id.split('__')[0]] = rank_id.split('__')[1]

In [100]:
get_accuracy_dict(mapping_results, query_tax_dict)

{'k': 1.0,
 'p': 0.7808393002809753,
 'c': 0.26216804133055377,
 'o': 0.2609444394090456,
 'f': 0.2585425541557147,
 'g': 0.10237469409951962,
 's': 4.5318589685488985e-05}